In [1]:
# Discriminator

import torch
import torch.nn as nn


class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride,
                      1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.inital = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2),
        )

				# Compile Disc Layers
        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(Block(in_channels, feature,
                          stride=1 if feature == features[-1] else 2))
            in_channels = feature
        layers.append(nn.Conv2d(in_channels, 1, kernel_size=4,
                      stride=1, padding=1, padding_mode="reflect"))
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
      x = self.inital(x)
      return torch.sigmoid(self.model(x))
        
def test():
  x = torch.randn((1, 3, 256, 256))
  model = Discriminator(in_channels=3)
  preds = model(x)
  print(preds.shape)
  
test()

torch.Size([1, 1, 30, 30])


In [2]:
# Generator

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs) if down else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.conv(x)


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False,
                      kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, img_channels, num_features=64, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size=7,
                      stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True)
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(num_features, num_features*2, down=True,
                          kernel_size=3, stride=2, padding=1),
                ConvBlock(num_features * 2, num_features * 4,
                          down=True, kernel_size=3, stride=2, padding=1),
            ]
        )

        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )

        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(num_features * 4, num_features * 2, down=False,
                          kernel_size=3, stride=2, padding=1, output_padding=1),
                ConvBlock(num_features * 2, num_features, down=False,
                          kernel_size=3, stride=2, padding=1, output_padding=1),
            ]
        )

        self.last = nn.Conv2d(num_features, img_channels, kernel_size=7,
                              stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.residual_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)

        return torch.tanh(self.last(x))


def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = Generator(img_channels, 9)
    print(gen(x).shape)


test()

torch.Size([2, 3, 256, 256])


In [3]:
# Utils

import random, torch, os, numpy as np
import torch.nn as nn
import copy
import albumentations as A
from albumentations.pytorch import ToTensorV2

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr, device):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
transforms = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        ToTensorV2(),
     ],
    additional_targets={"image0": "image"},
)

In [4]:
# Dataloader

from PIL import Image
import os
import numpy as np
from torch.utils.data import Dataset


class ABDataset(Dataset):
    def __init__(self, root_A, root_B, transform=None):
        self.root_A = root_A
        self.root_B = root_B
        self.transform = transform

        self.a_images = os.listdir(root_A)
        self.b_images = os.listdir(root_B)
        
        self.length_dataset = max(len(self.a_images), len(self.b_images))
        self.a_len = len(self.a_images)
        self.b_len = len(self.b_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, idx):
        a_img = self.a_images[idx % self.a_len]
        b_img = self.b_images[idx % self.b_len]
        a_pth = os.path.join(self.root_A, a_img)
        b_pth = os.path.join(self.root_B, b_img)
        a_img = np.array(Image.open(a_pth).convert("RGB"))
        b_img = np.array(Image.open(b_pth).convert("RGB"))
        if self.transform:
            augs = self.transform(image=a_img, image0=b_img)
            a_img = augs["image"]
            b_img = augs["image0"]
        
        return a_img, b_img


In [5]:
# Config + Training

from torchvision.datasets import ImageFolder
import torch.optim as optim
from torchvision.utils import save_image
from tqdm import tqdm
from torch.utils.data import DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "./../_datasets/cyclegan/horse2zebra/horse2zebra"
BATCH_SIZE = 1
LEARNING_RATE = 2e-4
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 0
NUM_EPOCHS = 150
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_B = "genB.pth.tar"  # B = Horse
CHECKPOINT_GEN_A = "genA.pth.tar"  # A = Zebra
CHECKPOINT_CRITIC_B = "criticB.pth.tar"
CHECKPOINT_CRITIC_A = "criticA.pth.tar"

def train_fn(disc_A, disc_B, gen_B, gen_A, loader, opt_disc, opt_gen, L1, mse, d_scaler, g_scaler, epoch):
    loop = tqdm(loader, leave=True)

    for idx, (a_img, b_img) in enumerate(loop):
        a_img = a_img.to(DEVICE)
        b_img = b_img.to(DEVICE)  # a_img = horse, b_img = zebra

        # Train Discriminators
        with torch.cuda.amp.autocast():
            fake_a = gen_B(b_img)
            D_A_real = disc_A(a_img)
            D_A_fake = disc_A(fake_a.detach())
            D_A_real_loss = mse(D_A_real, torch.ones_like(D_A_real))
            D_A_fake_loss = mse(D_A_fake, torch.zeros_like(D_A_fake))
            D_A_loss = D_A_real_loss + D_A_fake_loss

            fake_b = gen_A(a_img)
            D_B_real = disc_B(b_img)
            D_B_fake = disc_B(fake_b.detach())
            D_B_real_loss = mse(D_B_real, torch.ones_like(D_B_real))
            D_B_fake_loss = mse(D_B_fake, torch.zeros_like(D_B_fake))
            D_B_loss = D_B_real_loss + D_B_fake_loss

            # Put the loss together
            D_loss = (D_A_loss + D_B_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators
        with torch.cuda.amp.autocast():
            # Adversarial Loss for Both Generators
            D_A_fake = disc_A(fake_a)
            D_B_fake = disc_B(fake_b)
            loss_G_A = mse(D_A_fake, torch.ones_like(D_A_fake))
            loss_G_B = mse(D_B_fake, torch.ones_like(D_B_fake))

            # Cycle Loss
            cycle_b = gen_B(fake_a)
            cycle_a = gen_A(fake_b)
            cycle_b_loss = L1(b_img, cycle_b)
            cycle_a_loss = L1(a_img, cycle_a)

            # Identity Loss
            id_b = gen_B(b_img)
            id_a = gen_A(a_img)
            id_b_loss = L1(b_img, id_b)
            id_a_loss = L1(a_img, id_a)

            # Add together
            G_loss = (
                loss_G_B
                +
                loss_G_A
                +
                cycle_b_loss * LAMBDA_CYCLE
                +
                cycle_a_loss * LAMBDA_CYCLE
                +
                id_a_loss * LAMBDA_IDENTITY
                +
                id_b_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        loop.set_postfix(e=epoch, d_loss=D_loss.item(), g_loss=G_loss.item())
        # Print
        if idx % 100 == 0:
            save_image(torch.cat((a_img*0.5+0.5, b_img*0.5+0.5,fake_a*0.5+0.5, fake_b*0.5+0.5), 0), f"image/grid{epoch}-{idx}.png", nrow=2, normalize=True)


def main():
    disc_A = Discriminator(3).to(DEVICE)
    disc_B = Discriminator(3).to(DEVICE)
    gen_A = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    gen_B = Generator(img_channels=3, num_residuals=9).to(DEVICE)

    opt_disc = optim.Adam(
        list(disc_A.parameters()) + list(disc_B.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999)
    )

    opt_gen = optim.Adam(
        list(gen_A.parameters()) + list(gen_B.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999)
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_A, gen_A, opt_gen, LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_GEN_B, gen_B, opt_gen, LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_A, disc_A, opt_disc, LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_B, disc_B, opt_disc, LEARNING_RATE,
        )

    dataset = ABDataset(root_A="./../_datasets/cyclegan/horse2zebra/horse2zebra/trainB",
                        root_B="./../_datasets/cyclegan/horse2zebra/horse2zebra/trainA", transform=transforms)

    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True
    )

    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(disc_A, disc_B, gen_B, gen_A, loader, opt_disc,
                 opt_gen, L1, mse, d_scaler, g_scaler, epoch)

        if SAVE_MODEL:
            save_checkpoint(gen_A, opt_gen, CHECKPOINT_GEN_A)
            save_checkpoint(gen_B, opt_gen, CHECKPOINT_GEN_B)
            save_checkpoint(disc_A, opt_disc, CHECKPOINT_CRITIC_A)
            save_checkpoint(disc_B, opt_disc, CHECKPOINT_CRITIC_B)


main()


100%|██████████| 1334/1334 [05:16<00:00,  4.22it/s, d_loss=0.496, e=0, g_loss=5.1] 


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 1334/1334 [05:08<00:00,  4.33it/s, d_loss=0.362, e=1, g_loss=5.88]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 1334/1334 [05:08<00:00,  4.33it/s, d_loss=0.375, e=2, g_loss=4.27]


...

100%|██████████| 1334/1334 [06:37<00:00,  3.35it/s, d_loss=0.243, e=142, g_loss=2.78] 


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 1334/1334 [06:28<00:00,  3.43it/s, d_loss=0.162, e=143, g_loss=2.84] 


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


 55%|█████▍    | 728/1334 [04:07<05:44,  1.76it/s, d_loss=0.238, e=144, g_loss=3.19] 